In [2]:
import os
os.chdir('/home/jovyan')

In [3]:
from src.postgres import connect_to_postgres

**adult data types**

* age                int64
* workclas          object
* fnlwgt             int64
* education         object
* eduation_num       int64
* marital_status    object
* occupation        object
* relationship      object
* race              object
* gender            object
* capital_gain       int64
* capital_loss       int64
* hours_per_week     int64
* native_country    object
* income_label      object

In [4]:
con, cur = connect_to_postgres()

cur.execute("""
BEGIN;
ALTER TABLE adult ADD COLUMN _id SERIAL PRIMARY KEY;
ALTER TABLE adult ADD COLUMN target BOOLEAN;
COMMIT;
""")

con.close()

DuplicateColumn: column "_id" of relation "adult" already exists


In [41]:
con, cur = connect_to_postgres()

cur.execute("""
SELECT income_label FROM adult LIMIT 5;
""")

print(cur.fetchall())
con.close()

[(' >50K',), (' >50K',), (' >50K',), (' <=50K',), (' <=50K',)]


In [37]:
con, cur = connect_to_postgres()

cur.execute("""
SELECT _id, income_label FROM adult WHERE target IS NULL;
""")

this_id, income_label = cur.fetchone()
greater_than_50k = (income_label == ' >50K')

cur.execute("""
BEGIN;
UPDATE adult
SET target = {}
WHERE _id = {};
COMMIT;
""".format(greater_than_50k, this_id))

con.close()

True


In [38]:
con, cur = connect_to_postgres()

cur.execute("""
SELECT _id, income_label, target FROM adult WHERE _id = {};
""".format(this_id))
print(this_id, cur.fetchone())
con.close()

9 (9, ' >50K', True)


In [39]:
this_id

9

In [40]:
con, cur = connect_to_postgres()

cur.execute("""
SELECT _id, target FROM adult WHERE target IS NOT NULL;
""")

print(cur.fetchall())
con.close()

[(1, False), (3, False), (4, False), (5, False), (6, False), (7, False), (8, False), (9, True), (2, False)]


In [43]:
def encode_target(_id):
    """Encode the target for a single row as a boolean value. Takes a row _id."""
    
    con, cur = connect_to_postgres()
    
    cur.execute("""SELECT _id, income_label FROM adult WHERE _id = {};""".format(_id))

    this_id, income_label = cur.fetchone()
    
    assert this_id == _id
    
    greater_than_50k = (income_label == ' >50K')
    
    cur.execute("""
        BEGIN;
        UPDATE adult
        SET target = {}
        WHERE _id = {};
        COMMIT;
    """.format(greater_than_50k, _id))

    con.close()

In [44]:
encode_target(5)